In [1]:
'''creates column dictionary for Tascomi ingestion process'''

#from os import path, getenv
from pprint import pprint
from googleapiclient.discovery import build
#from googleapiclient.http import MediaIoBaseDownload
#from googleapiclient import errors, http
from oauth2client.service_account import ServiceAccountCredentials

SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

KEY_FILE_LOCATION = 'key_file.json'

GOOGLE_SHEET_ID='1ZZwWHSoudBgN9j0jV6ZrNZKgXYMOm7ObWTWLT3Xg8Rw'

credentials = ServiceAccountCredentials.from_json_keyfile_name(
    KEY_FILE_LOCATION,
    scopes=SCOPES)

service = build(
    'sheets',
    'v4',
    credentials=credentials,
    cache_discovery=False)

In [1]:
gsheet_workbook = service.spreadsheets().get(spreadsheetId=GOOGLE_SHEET_ID).execute()
sheets = gsheet_workbook.get('sheets')

rows = []
for sheet in sheets:
    data = service.spreadsheets().values().get(
            spreadsheetId=GOOGLE_SHEET_ID,
            range=sheet['properties']['title'],
            majorDimension='ROWS'
        ).execute()

    for i in data['values']:
        i.append(sheet['properties']['title'])

    rows.extend(data['values'][1:])

type_map={'real': 'float',
          'double precision': 'double',
          'bigint': 'long',
          'timestamp without time zone': 'timestamp'}

rows_types_replaced = []
for lst in rows:
    if lst[1] in type_map:
        lst[1]=type_map.get(lst[1])
        rows_types_replaced.append(lst)
    else:
        rows_types_replaced.append(lst)

types_to_include=['integer', 'timestamp', 'boolean', 'float', 'double', 'long']

rows_for_col_dict=[]
for lst in rows_types_replaced:
    if lst[1] in types_to_include:
        rows_for_col_dict.append(lst)
    else:
        pass

result = dict()
for value, key1, key2 in rows_for_col_dict:
    result.setdefault(key1, {}).setdefault(key2, list()).append(value)

pprint(result)